In [1]:
import torch
from torchvision import models

In [2]:
alexnet = models.alexnet(weights=models.AlexNet_Weights.DEFAULT)
preprocess = models.AlexNet_Weights.DEFAULT.transforms()
category = models.AlexNet_Weights.DEFAULT.meta["categories"]

In [3]:
from PIL import Image
owl = Image.open("../../data/owl.JPG")

In [4]:
owl_t = preprocess(owl)
owl_t.shape

torch.Size([3, 224, 224])

In [5]:
# モデルは(batch size, channel, H, W)の4次元を求めるため、unsqueeze()で拡張
batch_t = torch.unsqueeze(owl_t, dim=0)

In [6]:
alexnet.eval()
with torch.no_grad():
    output = alexnet(batch_t)

In [7]:
proba = output[0]
pred = torch.argmax(output[0]).item()
print(category[pred])
print("accuracy: ", proba[pred].item())


diamondback
accuracy:  8.328636169433594
